Its very important in retail business to understand customer buying pattern and association of items which are brought by customer. This project is to do market basket analysis/Association Analysis. 

This Code reads file from 
1. S3 containing customernumber (INT), Itemnbr(INT), Qty(INT). 
2. Used Jupiter to read the file and uses apriori for association analysis
3. Association analysis result are the stored in file and put back in amazon S3.

In [1]:
import csv
import pandas as pd
import gzip
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
import boto3
from sqlalchemy import create_engine
import pandas as pd
connstr='<enter connection string>'
engine =create_engine(connstr)


Below is to connect to AWS S3 and connect to S3. 
Data in File is in format of customernumber (INT), Itemnbr(INT), Qty(INT)

In [2]:
#Fetch Data from amazon S3 using boto3 Libarary 

bucket = "<AWS Bucket Name>"
file_name = "<File Name>"

s3 = boto3.client('s3') 
# 's3' is a key word. create connection to S3 using default config and all buckets within S3

obj = s3.get_object(Bucket= bucket, Key= file_name) 
# get object and file (key) from bucket

dataset = pd.read_csv(obj['Body'], compression='gzip', header=0, sep=',', quotechar='"',encoding="utf-8-sig")

In [9]:
# Test the dataset is fetching data correctly
dataset.head()

,seqnbr,itemnbr,quantity
0,628770236,671015,1
1,628770236,415500,1
2,628770153,180498,1
3,628770005,798971,1
4,628770005,724887,1


In [10]:
# print the Headers of the dataset
print(dataset.columns.tolist())

['seqnbr', 'itemnbr', 'quantity']


In [11]:
#prepare data step 1: creating basket
basket = (dataset.groupby(['seqnbr', 'itemnbr'])['quantity'].sum().unstack().reset_index().fillna(0).set_index('seqnbr'))

In [12]:
# Test the basket data sample
basket.head()

itemnbr,100573,101397,103135,105972,106206,106586,109293,109393,110848,112572,...,905086,907234,907429,907491,907623,908597,910024,912089,912229,912249
seqnbr,,,,,,,,,,,,,,,,,,,,,
628518360,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
628518362,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
628518711,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
628518715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
628518720,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [13]:
#prepare data step 2: 

def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [14]:
#using apriori for market basket and getting item feq which are brough together
frequent_itemsets = apriori(basket_sets, min_support=0.0007, use_colnames=True)

In [15]:
# Test the frequent_itemsets sample
frequent_itemsets.head()

,support,itemsets
0,0.001133,[100573]
1,0.003398,[101397]
2,0.010193,[103135]
3,0.001133,[105972]
4,0.006795,[106206]


In [16]:
# filter data for min_threshold =1.1
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.1)

In [17]:
rules["antecedant_len"] = rules["antecedants"].apply(lambda x: len(x))
rules["consequents_len"] = rules["consequents"].apply(lambda x: len(x))

rules[ (rules['antecedant_len'] == 1) & (rules["consequents_len"]==1)&
       (rules['confidence'] > 0.75) &
       (rules['lift'] > 1.2) ]

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedant_len,consequents_len
1,(100573),(101397),0.001133,0.003398,0.001133,1.0,294.333333,0.001129,inf,1,1
3,(100573),(142264),0.001133,0.032843,0.001133,1.0,30.448276,0.001095,inf,1,1
5,(100573),(166476),0.001133,0.002265,0.001133,1.0,441.500000,0.001130,inf,1,1
7,(100573),(168523),0.001133,0.012458,0.001133,1.0,80.272727,0.001118,inf,1,1
9,(100573),(351504),0.001133,0.069083,0.001133,1.0,14.475410,0.001054,inf,1,1
10,(100573),(671015),0.001133,0.097395,0.001133,1.0,10.267442,0.001022,inf,1,1
62,(696050),(106206),0.001133,0.006795,0.001133,1.0,147.166667,0.001125,inf,1,1
78,(109293),(639287),0.001133,0.052095,0.001133,1.0,19.195652,0.001074,inf,1,1
137,(887863),(115234),0.001133,0.006795,0.001133,1.0,147.166667,0.001125,inf,1,1
142,(158226),(115771),0.001133,0.003398,0.001133,1.0,294.333333,0.001129,inf,1,1


In [18]:
# Test the rules dataset sample
rules.head()

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedant_len,consequents_len
0,(101397),(100573),0.003398,0.001133,0.001133,0.333333,294.333333,0.001129,1.498301,1,1
1,(100573),(101397),0.001133,0.003398,0.001133,1.000000,294.333333,0.001129,inf,1,1
2,(142264),(100573),0.032843,0.001133,0.001133,0.034483,30.448276,0.001095,1.034541,1,1
3,(100573),(142264),0.001133,0.032843,0.001133,1.000000,30.448276,0.001095,inf,1,1
4,(166476),(100573),0.002265,0.001133,0.001133,0.500000,441.500000,0.001130,1.997735,1,1


In [4]:
#Create csv file
#rules[ (rules['antecedant_len'] == 1) & (rules["consequents_len"]==1)].to_csv('test.csv',sep=',')
# Create an S3 client
s3 = boto3.client('s3')

filename = '<file Name>'
bucket_name = '<Bucket Name>'

# Uploads the given file using a managed uploader, which will split up large
# files automatically and upload parts in parallel.
s3.upload_file(filename, bucket_name, filename)